In [21]:
pip install chess

# Changes
- Use MSE in GAN compilation instead of MAE
- Change alpha to 0.2 in discriminator and encoder of LeakyReLU
- I attempted to add another encoder and decoder, but I couldn't fix the data shaping problems
- resolve new_chess_dictionary by adding a function to get the pieces

Of any of the changes I make, it doesn't seem to be minimizing loss.  However, when comparing my output to the output of the original code, my board is much closer to something that would be real.  The original code had some output where one color had multiple bishops and queens.  Obviously this is not possible.  With my output, the board layout is a possible chess position.

# **Step 1| Prepare Data**
1. Download the data

In [22]:
!wget https://www.pgnmentor.com/players/Carlsen.zip
!unzip Carlsen.zip

--2021-11-19 19:17:19--  https://www.pgnmentor.com/players/Carlsen.zip
Resolving www.pgnmentor.com (www.pgnmentor.com)... 65.254.227.240
Connecting to www.pgnmentor.com (www.pgnmentor.com)|65.254.227.240|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 900358 (879K) [application/zip]
Saving to: ‘Carlsen.zip.1’

Carlsen.zip.1       100%[===================>] 879.26K  5.45MB/s    in 0.2s    

2021-11-19 19:17:20 (5.45 MB/s) - ‘Carlsen.zip.1’ saved [900358/900358]

Archive:  Carlsen.zip
replace Carlsen.pgn? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: Carlsen.pgn             


In [23]:
import chess.pgn
pgn = open("Carlsen.pgn")
sides = []
games = []
length = 100 # number of games
for i in range(length):
    try:
        if chess.pgn.read_game(pgn).mainline_moves():
            games.append(chess.pgn.read_game(pgn).mainline_moves())
            sides.append(chess.pgn.read_game(pgn).headers["White"])
    except:
        print(i,chess.pgn.read_game(pgn))
        pass
len(games)

100

In [24]:
X = []
y = []
counter2 = 0
for game in games:
    board = chess.Board()
    white = sides[counter2]
    if white == 'Carlsen,Magnus':
        remainder = 0
    else:
        remainder = 1
    counter = 0
    for move in game:
        if counter % 2 == remainder:
            X.append(board.copy())
        board.push(move)
        if counter % 2 == remainder:
            y.append(board.copy())
        counter += 1
    counter2 += 1

In [25]:
chess_dict = {
    'p' : [1,0,0,0,0,0,0,0,0,0,0,0,0],
    'P' : [0,0,0,0,0,0,1,0,0,0,0,0,0],
    'n' : [0,1,0,0,0,0,0,0,0,0,0,0,0],
    'N' : [0,0,0,0,0,0,0,1,0,0,0,0,0],
    'b' : [0,0,1,0,0,0,0,0,0,0,0,0,0],
    'B' : [0,0,0,0,0,0,0,0,1,0,0,0,0],
    'r' : [0,0,0,1,0,0,0,0,0,0,0,0,0],
    'R' : [0,0,0,0,0,0,0,0,0,1,0,0,0],
    'q' : [0,0,0,0,1,0,0,0,0,0,0,0,0],
    'Q' : [0,0,0,0,0,0,0,0,0,0,1,0,0],
    'k' : [0,0,0,0,0,1,0,0,0,0,0,0,0],
    'K' : [0,0,0,0,0,0,0,0,0,0,0,1,0],
    '.' : [0,0,0,0,0,0,0,0,0,0,0,0,1],
}
def make_matrix(board): 
    pgn = board.epd()
    foo = []  
    pieces = pgn.split(" ", 1)[0]
    rows = pieces.split("/")
    for row in rows:
        foo2 = []  
        for thing in row:
            if thing.isdigit():
                for i in range(0, int(thing)):
                    foo2.append('.')
            else:
                foo2.append(thing)
        foo.append(foo2)
    return foo
def translate(matrix,chess_dict):
    rows = []
    for row in matrix:
        terms = []
        for term in row:
            terms.append(chess_dict[term])
        rows.append(terms)
    return rows
import numpy as np
for i in range(len(X)):
    X[i] = translate(make_matrix(X[i]),chess_dict)
for i in range(len(y)):
    y[i] = translate(make_matrix(y[i]),chess_dict)
X = np.array(X)
y = np.array(y)

# **Step 2| MagnusGAN:**

In [26]:
from numpy import expand_dims
from numpy import zeros
from numpy import ones
from numpy import vstack
from numpy.random import randn
from numpy.random import randint
from keras.models import Model
from keras.layers import Input
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers.convolutional import Conv2D,Conv2DTranspose
from keras.layers.pooling import MaxPooling2D
from keras.layers.merge import concatenate
from keras.initializers import RandomNormal
from keras.layers import LeakyReLU
from keras.layers import BatchNormalization
from keras.layers import Activation,Reshape
from tensorflow.keras.optimizers import Adam
from keras.models import Sequential
from keras.layers import Dropout

In [27]:
def define_discriminator():
    init = RandomNormal(stddev=0.02)
    in_src_image = Input(shape=image_shape)
    in_target_image = Input(shape=image_shape)
    merged = concatenate([in_src_image, in_target_image])
    #image multiplied with kernel (64, (4,4))
    #64 is the num of node layers
    d = Conv2D(64, (4,4), strides=(2,2), padding='same', kernel_initializer=init)(merged)
    d = LeakyReLU(alpha=0.2)(d)
    d = Conv2D(128, (4,4), strides=(2,2), padding='same', kernel_initializer=init)(d)
    d = BatchNormalization()(d)
    d = LeakyReLU(alpha=0.2)(d)
    d = Conv2D(256, (4,4), strides=(2,2), padding='same', kernel_initializer=init)(d)
    d = BatchNormalization()(d)
    d = LeakyReLU(alpha=0.2)(d)
    d = Conv2D(512, (4,4), strides=(2,2), padding='same', kernel_initializer=init)(d)
    d = BatchNormalization()(d)
    d = LeakyReLU(alpha=0.2)(d)
    d = Conv2D(512, (4,4), padding='same', kernel_initializer=init)(d)
    d = BatchNormalization()(d)
    d = LeakyReLU(alpha=0.2)(d)
    d = Conv2D(1, (4,4), padding='same', kernel_initializer=init)(d)
    patch_out = Activation('sigmoid')(d)
    model = Model(inputs = [in_src_image, in_target_image], outputs = patch_out)
    opt = Adam(lr=0.0002, beta_1=0.5)
    model.compile(loss='binary_crossentropy', optimizer=opt, loss_weights=[0.5])
    return model

In [28]:
# reduce image size
def define_encoder_block(layer_in, n_filters, batchnorm=True):
    init = RandomNormal(stddev=0.02)
    g = Conv2D(n_filters, (4,4), strides=(2,2), padding='same', kernel_initializer=init)(layer_in)
    if batchnorm:
        g = BatchNormalization()(g, training=True)
    # can change alpha
    g = LeakyReLU(alpha=0.2)(g)
    return g
 
 # increase image size
def decoder_block(layer_in, skip_in, n_filters, dropout=True):
    init = RandomNormal(stddev=0.02)
    g = Conv2DTranspose(n_filters, (4,4), strides=(2,2), padding='same', kernel_initializer=init)(layer_in)
    g = BatchNormalization()(g, training=True)
    if dropout:
        # can change dropout
        g = Dropout(0.5)(g, training=True)
    g = concatenate([g, skip_in])
    g = Activation('relu')(g)
    return g

In [29]:
def define_generator(image_shape=(8,8,13)):
    init = RandomNormal(stddev=0.02)
    in_image = Input(shape=image_shape)
    # 2 encoders e1 and e2, can encode 2x, 3x, etc?
    e1 = define_encoder_block(in_image, 64, batchnorm=False)
    e2 = define_encoder_block(e1, 128)
    b = Conv2D(512, (4,4), strides=(2,2), padding='same', kernel_initializer=init)(e2)
    b = Activation('relu')(b)
    d6 = decoder_block(b, e2, 128, dropout=False)
    d7 = decoder_block(d6, e1, 64, dropout=False)
    g = Conv2DTranspose(13, (4,4), strides=(2,2), padding='same', kernel_initializer=init)(d7)
    out_image = Activation('softmax')(g)
    model = Model(in_image, out_image)
    return model

In [30]:
# g is generator, d is discriminator, weights change to reduce loss
def define_gan(g_model, d_model, image_shape):
    d_model.trainable = False
    in_src = Input(shape=image_shape)
    gen_out = g_model(in_src)
    dis_out = d_model([in_src, gen_out])
    model = Model(in_src, [dis_out, gen_out])
    opt = Adam(lr=0.0002, beta_1=0.5)
    # can use mse
    model.compile(loss=['binary_crossentropy', 'mse'], optimizer=opt, loss_weights=[1,100])
    return model

# **Step 3| Prepare for execution:**

In [31]:
def generate_real_samples(dataset, n_samples, patch_shape):
    trainA, trainB = dataset
    ix = randint(0, trainA.shape[0], n_samples)
    X1, X2 = trainA[ix], trainB[ix]
    y = ones((n_samples, patch_shape, patch_shape, 1))
    return [X1, X2], y
 
def generate_fake_samples(g_model, samples, patch_shape):
    X = g_model.predict(samples)
    y = zeros((len(X), patch_shape, patch_shape, 1))
    return X, y

In [32]:
def train(d_model, g_model, gan_model, dataset, n_epochs=1, n_batch=1):
    n_patch = d_model.output_shape[1]
    trainA, trainB = dataset
    bat_per_epo = int(len(trainA) / n_batch)
    n_steps = bat_per_epo * n_epochs
    for i in range(n_steps):
        [X_realA, X_realB], y_real = generate_real_samples(dataset, n_batch, n_patch)
        X_fakeB, y_fake = generate_fake_samples(g_model, X_realA, n_patch)
        d_loss1 = d_model.train_on_batch([X_realA, X_realB], y_real)
        d_loss2 = d_model.train_on_batch([X_realA, X_fakeB], y_fake)
        g_loss, _, _ = gan_model.train_on_batch(X_realA, [y_real, X_realB])
        print('>%d, d1[%.3f] d2[%.3f] g[%.3f]' % (i+1, d_loss1, d_loss2, g_loss))
    if (i+1) % (bat_per_epo * 10) == 0:
        clear_output()

In [33]:
image_shape = (8,8,13)
d_model = define_discriminator()
g_model = define_generator()
gan_model = define_gan(g_model, d_model, image_shape)
train(d_model, g_model, gan_model, [X,y])

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


>1, d1[0.347] d2[0.347] g[7.865]
>2, d1[0.346] d2[0.347] g[7.820]
>3, d1[0.346] d2[0.347] g[7.815]
>4, d1[0.346] d2[0.347] g[7.764]
>5, d1[0.346] d2[0.347] g[7.715]
>6, d1[0.346] d2[0.347] g[7.724]
>7, d1[0.346] d2[0.347] g[7.655]
>8, d1[0.346] d2[0.347] g[7.571]
>9, d1[0.346] d2[0.347] g[7.553]
>10, d1[0.346] d2[0.347] g[7.505]
>11, d1[0.346] d2[0.347] g[7.430]
>12, d1[0.346] d2[0.347] g[7.412]
>13, d1[0.346] d2[0.347] g[7.341]
>14, d1[0.346] d2[0.347] g[7.299]
>15, d1[0.346] d2[0.347] g[7.201]
>16, d1[0.346] d2[0.347] g[7.124]
>17, d1[0.346] d2[0.347] g[7.100]
>18, d1[0.346] d2[0.347] g[7.102]
>19, d1[0.346] d2[0.347] g[7.014]
>20, d1[0.346] d2[0.347] g[6.956]
>21, d1[0.346] d2[0.347] g[6.885]
>22, d1[0.346] d2[0.347] g[6.829]
>23, d1[0.346] d2[0.347] g[6.781]
>24, d1[0.346] d2[0.347] g[6.776]
>25, d1[0.346] d2[0.347] g[6.753]
>26, d1[0.347] d2[0.347] g[6.575]
>27, d1[0.347] d2[0.347] g[6.461]
>28, d1[0.347] d2[0.347] g[6.869]
>29, d1[0.347] d2[0.347] g[6.356]
>30, d1[0.347] d2[0.347

# **Step 4| Observe results:**

In [37]:
import random

# create a way to get the piece in each spot, resolve new_chess_dict undef 
def get_piece(val):
    for key, value in chess_dict.items():
         if np.array_equal(val, value):
             return key 
    return "Unsure"

flatten = lambda l: [item for sublist in l for item in sublist]
instance = random.randint(1,len(X)-1)
state = X[instance].reshape(1,8,8,13)
action = gan_model.predict(state)[1]
def retranslate(action):
    board = []
    flatten_action = flatten(flatten(action))
    for i in range(len(flatten_action)):
        new_set = np.zeros((13,))
        max_index = list(flatten_action[i]).index(max(flatten_action[i]))
        new_set[max_index] = 1
        board.append(new_set)
    for i in range(len(board)):
        # print(board[i])
        board[i] = get_piece(board[i])
    board = np.array(board).reshape(8,8)
    print(board)
        
retranslate(action)

[['r' 'n' 'b' 'q' 'k' 'b' '.' 'r']
 ['p' 'p' 'p' 'p' '.' 'p' 'p' 'p']
 ['.' '.' '.' '.' '.' 'n' '.' '.']
 ['.' '.' '.' '.' 'p' '.' '.' '.']
 ['.' '.' '.' '.' 'P' '.' '.' '.']
 ['.' '.' '.' '.' '.' 'N' '.' '.']
 ['P' 'P' 'P' 'P' '.' 'P' 'P' 'P']
 ['R' 'N' 'B' 'Q' 'K' 'B' '.' 'R']]
